In [22]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [26]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T

spark = SparkSession.builder.appName("Dataframe Joins - UFC data").getOrCreate()

<IPython.core.display.Javascript object>

In [27]:
spark

<IPython.core.display.Javascript object>

In [1]:
import os
from os.path import isfile, join

In [18]:
loc = os.path.abspath("")
data_loc = f"{loc}/data/ufc/"
data_files = [f for f in os.listdir(data_loc) if f.endswith('.csv')]

In [19]:
data_files

['data.csv',
 'preprocessed_data.csv',
 'raw_fighter_details.csv',
 'raw_total_fight_data.csv']

In [30]:
data = {}
for file in data_files:
    key = file.split('.csv')[0]
    data[key] = spark.read.csv(f"{data_loc}{file}", inferSchema=True, header=True)
    data[key].createOrReplaceTempView(f"data_{key}")

print(data.keys())

dict_keys(['data', 'preprocessed_data', 'raw_fighter_details', 'raw_total_fight_data'])


<IPython.core.display.Javascript object>

In [35]:
data['data'].columns

['R_fighter',
 'B_fighter',
 'Referee',
 'date',
 'location',
 'Winner',
 'title_bout',
 'weight_class',
 'no_of_rounds',
 'B_current_lose_streak',
 'B_current_win_streak',
 'B_draw',
 'B_avg_BODY_att',
 'B_avg_BODY_landed',
 'B_avg_CLINCH_att',
 'B_avg_CLINCH_landed',
 'B_avg_DISTANCE_att',
 'B_avg_DISTANCE_landed',
 'B_avg_GROUND_att',
 'B_avg_GROUND_landed',
 'B_avg_HEAD_att',
 'B_avg_HEAD_landed',
 'B_avg_KD',
 'B_avg_LEG_att',
 'B_avg_LEG_landed',
 'B_avg_PASS',
 'B_avg_REV',
 'B_avg_SIG_STR_att',
 'B_avg_SIG_STR_landed',
 'B_avg_SIG_STR_pct',
 'B_avg_SUB_ATT',
 'B_avg_TD_att',
 'B_avg_TD_landed',
 'B_avg_TD_pct',
 'B_avg_TOTAL_STR_att',
 'B_avg_TOTAL_STR_landed',
 'B_longest_win_streak',
 'B_losses',
 'B_avg_opp_BODY_att',
 'B_avg_opp_BODY_landed',
 'B_avg_opp_CLINCH_att',
 'B_avg_opp_CLINCH_landed',
 'B_avg_opp_DISTANCE_att',
 'B_avg_opp_DISTANCE_landed',
 'B_avg_opp_GROUND_att',
 'B_avg_opp_GROUND_landed',
 'B_avg_opp_HEAD_att',
 'B_avg_opp_HEAD_landed',
 'B_avg_opp_KD',
 'B_av

<IPython.core.display.Javascript object>

In [62]:
# Onley data from 2017+

data["data"] = (
    data["data"]
    .where(F.col("date") > "2017-01-01")
    .withColumn(
        "Winner_name",
        F.when(F.col("Winner") == "Red", F.col("R_fighter")).otherwise(
            F.col("B_fighter")
        ),
    )
    .withColumn(
        "Losser_name",
        F.when(F.col("Winner") == "Red", F.col("B_fighter")).otherwise(
            F.col("R_fighter")
        ),
    )
)

<IPython.core.display.Javascript object>

In [63]:
data['data'].select('Winner_name', "Losser_name").show(5)

+--------------------+--------------+
|         Winner_name|   Losser_name|
+--------------------+--------------+
|        Henry Cejudo| Marlon Moraes|
|Valentina Shevchenko|   Jessica Eye|
|       Tony Ferguson|Donald Cerrone|
|            Petr Yan| Jimmie Rivera|
|       Blagoy Ivanov|   Tai Tuivasa|
+--------------------+--------------+
only showing top 5 rows



<IPython.core.display.Javascript object>

In [65]:
losses = (
    data["data"]
    .withColumnRenamed("Losser_name", "name")
    .groupBy("name")
    .count()
    .select("name", F.expr("count as losses"))
)

wins = (
    data["data"]
    .withColumnRenamed("Winner_name", "name")
    .groupBy("name")
    .count()
    .select("name", F.expr("count as wins"))
)

<IPython.core.display.Javascript object>

In [45]:
wins.show()

+-------------------+----+
|               name|wins|
+-------------------+----+
|        Chad Mendes|   1|
|          Zak Ottow|   3|
|Deiveson Figueiredo|   3|
|       Marcelo Golm|   1|
|           Jon Tuck|   1|
|    Danielle Taylor|   1|
|     Montel Jackson|   2|
| Ovince Saint Preux|   4|
| Alexander Yakovlev|   1|
|         Polo Reyes|   1|
|     Corey Anderson|   3|
|   Stephen Thompson|   1|
|     Augusto Mendes|   1|
|       Matt Schnell|   3|
|      Gilbert Burns|   4|
|        Josh Emmett|   3|
|         Juan Adams|   1|
|      Ricardo Lamas|   2|
|       Alex Caceres|   2|
|          Kevin Lee|   3|
+-------------------+----+
only showing top 20 rows



<IPython.core.display.Javascript object>

In [71]:
# We required to know the quantity of winners and lossers to make a correct join

red = data["data"].select("R_fighter").distinct()
blue = data["data"].select("B_fighter").distinct()
check = red.union(blue).distinct().count()
check

763

<IPython.core.display.Javascript object>

In [72]:
print(wins.distinct().count())
print(losses.distinct().count())

535
645


<IPython.core.display.Javascript object>

In [73]:
# Inner join
win_loss_data = wins.join(losses, "name")
print(win_loss_data.distinct().count())

# No es útil en este caso. Esta es la intersección. Se incluyen solo aquellos que ganaron y perdieron

417


<IPython.core.display.Javascript object>

In [74]:
# Left join
win_loss_data = wins.join(losses, "name", "left")
print(win_loss_data.distinct().count())

# No es útil en este caso. Solo incluye aquellos que hayan ganado almenos una vez


535


<IPython.core.display.Javascript object>

In [75]:
# Right join
win_loss_data = wins.join(losses, "name", "right")
print(win_loss_data.distinct().count())

# No es útil en este caso. Solo incluye aquellos que hayan perdido almenos una vez

645


<IPython.core.display.Javascript object>

In [84]:
# Outer join
win_loss_data = wins.join(losses, "name", "outer")
print(win_loss_data.distinct().count())


763


<IPython.core.display.Javascript object>

In [77]:
# More wins

win_loss_data.orderBy(F.desc("wins")).show(5)

+--------------------+----+------+
|                name|wins|losses|
+--------------------+----+------+
|       Thiago Santos|   8|     1|
|     Israel Adesanya|   6|  null|
|Alexander Volkano...|   6|  null|
|Anthony Rocco Martin|   6|     1|
|   Aljamain Sterling|   6|     2|
+--------------------+----+------+
only showing top 5 rows



<IPython.core.display.Javascript object>

In [85]:
# Youngest fighter that never lost
win_loss_data = win_loss_data.join(
    data["raw_fighter_details"].withColumnRenamed("fighter_name", "name"),
    "name",
    "left"
)

<IPython.core.display.Javascript object>

In [86]:
win_loss_data.select("DOB").show(5)

+------------+
|         DOB|
+------------+
|May 01, 1985|
|Apr 20, 1991|
|Aug 11, 1986|
|Oct 25, 1989|
|Dec 18, 1987|
+------------+
only showing top 5 rows



<IPython.core.display.Javascript object>

In [87]:
win_loss_data = (
    win_loss_data.withColumn("birthday", F.to_date(F.col("DOB"), "MMM dd, yyyy"))
    .withColumn(
        "age", F.round(F.months_between(F.current_date(), F.col("birthday")) / 12, 1)
    )
    .select("name", "wins", "losses", "age")
)

<IPython.core.display.Javascript object>

In [88]:
win_loss_data.show(5)

+-------------------+----+------+----+
|               name|wins|losses| age|
+-------------------+----+------+----+
|        Chad Mendes|   1|     1|35.6|
|        Kyle Nelson|null|     2|29.7|
|        Artem Lobov|null|     3|34.4|
|    Danielle Taylor|   1|     2|31.1|
|Deiveson Figueiredo|   3|     1|33.0|
+-------------------+----+------+----+
only showing top 5 rows



<IPython.core.display.Javascript object>

In [93]:
win_loss_data.where(F.col("losses").isNull()).orderBy(
    F.asc("age"), F.desc("wins")
).where("wins > 5").show(5)

+--------------------+----+------+----+
|                name|wins|losses| age|
+--------------------+----+------+----+
|     Israel Adesanya|   6|  null|31.4|
|      Dustin Poirier|   6|  null|31.9|
|Alexander Volkano...|   6|  null|32.2|
|        Kamaru Usman|   6|  null|33.6|
+--------------------+----+------+----+



<IPython.core.display.Javascript object>